In [34]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import getpass
import os
import shutil
import ipywidgets as widgets
from IPython.display import display, HTML

In [2]:
# Prompt user for AGOL credentials
username = input("Enter your AGOL username: ")
password = getpass.getpass("Enter your AGOL password: ")

# Connect to AGOL
gis = GIS("https://www.arcgis.com", username, password)


In [31]:

# Provide the AGOL item ID
item_id = "caaecc422e244897ad5a9d5e0ecf156b"

# Get the item
item = gis.content.get(item_id)

# Create a dropdown widget for selecting the layer
layer_options = [(layer.properties.name, i) for i, layer in enumerate(item.layers)]
layer_dropdown = widgets.Dropdown(
    options=layer_options,
    description='Select Layer:',
    disabled=False,
)

def on_layer_change(change):
    global feature_layer
    feature_layer = item.layers[change['new']]
    print(f"Selected layer: {feature_layer.properties.name}")

layer_dropdown.observe(on_layer_change, names='value')

# Display the dropdown widget
display(layer_dropdown)

# Create a directory to save attachments
output_dir = "attachments"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Dropdown(description='Select Layer:', options=(('admin_0', 0), ('extreme_events', 1)), value=0)

Selected layer: extreme_events


In [ ]:

# Function to download attachments
def download_attachments(feature_layer, output_dir):


    features = feature_layer.query(where="1=1", out_fields="*").features

    #print the index of the feature
    # Initialize an empty list to store feature data
    data = []

    for feature in features:
        object_id = feature.attributes['objectid']
        country_label = feature.attributes['country_label_2']
        globalid = feature.attributes['globalid']
        attachments = feature_layer.attachments.get_list(object_id)
        
        if attachments:
            for attachment in attachments:
                attachment_id = attachment['id']
                parentGlobalid = attachment['parentGlobalId']
                attachment_name= attachment['name']
                final_attachment_path = os.path.join(output_dir, f"{country_label}_{globalid}_{attachment_name}")
            
                # Check if the attachment already exists
                if not os.path.exists(final_attachment_path):
                    temp_dir = os.path.join(output_dir, f"temp_{object_id}")
                    
                    # Create a temporary directory to download the attachment
                    if not os.path.exists(temp_dir):
                        os.makedirs(temp_dir)
                    
                    # Download the attachment to the temporary directory
                    feature_layer.attachments.download(oid=object_id, attachment_id=attachment_id, save_path=temp_dir)
                    
                    # Move the attachment from the temporary directory to the output directory with the new name
                    temp_attachment_path = os.path.join(temp_dir, attachment_name)
                    shutil.move(temp_attachment_path, final_attachment_path)
                    
                    # Remove the temporary directory
                    shutil.rmtree(temp_dir)
                    
                    print(f"Downloaded {attachment_name} for feature {object_id} as {final_attachment_path}")
                else:
                    print(f"Attachment {attachment_name} for feature {object_id} already exists as {final_attachment_path}")
                
                # Append the feature data to the list
                data.append({
                    'object_id': object_id,
                    'country_label': country_label,
                    'globalid': globalid,
                    'final_attachment_path': final_attachment_path
                })
        else:
            # Append the feature data with no attachments
            data.append({
                'object_id': object_id,
                'country_label': country_label,
                'globalid': globalid,
                'final_attachment_path': None
            })
    return data

data = download_attachments(feature_layer, output_dir)

# Create a DataFrame from the list
df = pd.DataFrame(data)
df.head()


Attachment Screenshot_20240614-221010.jpg for feature 194 already exists as attachments\Côte d'Ivoire_3e0ac251-71a8-488f-8deb-bce2e3ea84e0_Screenshot_20240614-221010.jpg
Attachment Screenshot_20240614-180234.jpg for feature 194 already exists as attachments\Côte d'Ivoire_3e0ac251-71a8-488f-8deb-bce2e3ea84e0_Screenshot_20240614-180234.jpg
Attachment Screenshot_20240616-180857.jpg for feature 195 already exists as attachments\Côte d'Ivoire_13b05cad-9f1b-4ad0-8500-620fbbb1fb2a_Screenshot_20240616-180857.jpg
Attachment Screenshot_20240615-110620.jpg for feature 195 already exists as attachments\Côte d'Ivoire_13b05cad-9f1b-4ad0-8500-620fbbb1fb2a_Screenshot_20240615-110620.jpg
Attachment press_20220914.pdf for feature 211 already exists as attachments\Japan_1a0ef617-0b09-46a5-8c73-f19f37415a4b_press_20220914.pdf
Attachment press_20230928.pdf for feature 211 already exists as attachments\Japan_1a0ef617-0b09-46a5-8c73-f19f37415a4b_press_20230928.pdf


,object_id,country_label,globalid,final_attachment_path
0,194,Côte d'Ivoire,3e0ac251-71a8-488f-8deb-bce2e3ea84e0,attachments\Côte d'Ivoire_3e0ac251-71a8-488f-8...
1,194,Côte d'Ivoire,3e0ac251-71a8-488f-8deb-bce2e3ea84e0,attachments\Côte d'Ivoire_3e0ac251-71a8-488f-8...
2,195,Côte d'Ivoire,13b05cad-9f1b-4ad0-8500-620fbbb1fb2a,attachments\Côte d'Ivoire_13b05cad-9f1b-4ad0-8...
3,195,Côte d'Ivoire,13b05cad-9f1b-4ad0-8500-620fbbb1fb2a,attachments\Côte d'Ivoire_13b05cad-9f1b-4ad0-8...
4,197,United Kingdom of Great Britain and Northern I...,7664f9ce-112b-44af-8ace-945b84ccd1be,None


In [ ]:
# Save the DataFrame to an Excel file
output_excel_path = os.path.join(output_dir, 'features_with_attachments.xlsx')
df.to_excel(output_excel_path, index=True, engine='xlsxwriter')

# Load the workbook and worksheet to embed images
with pd.ExcelWriter(output_excel_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, index=False)
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    
    # Widen the first column to make the caption clearer
    worksheet.set_column(0, 0, 30)
    worksheet.write(0, 0, "Embed images that scale to cell size")
    
    # Embed images in the final_attachment_path column
    for row_num, attachment_path in enumerate(df['final_attachment_path'], start=1):
        if attachment_path:
            #adjust attachment path to be fully qualified path
            attachment_path = os.path.abspath(attachment_path)
            print(f"Embedding image {attachment_path} in row {row_num}")
            
            # check for file extension embed images as jpg, jpeg, png using embed_image and hyperlink other file types
            if attachment_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                worksheet.set_row(row_num, 60)  # Adjust row height
                worksheet.embed_image(row_num, df.columns.get_loc('final_attachment_path'), attachment_path, {'x_scale': 0.5, 'y_scale': 0.5})

            else:
                # Add a hyperlink to the file
                worksheet.write_url(row_num, df.columns.get_loc('final_attachment_path'), f'external:{attachment_path}', string='Link to attachment')


print(f"DataFrame saved as Excel file with embedded images at {output_excel_path}")

Embedding image c:\Users\rami8629\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\Code-Repos\ArcGIS_Code_Repo\src\tests\attachments\Côte d'Ivoire_3e0ac251-71a8-488f-8deb-bce2e3ea84e0_Screenshot_20240614-221010.jpg in row 1
Embedding image c:\Users\rami8629\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\Code-Repos\ArcGIS_Code_Repo\src\tests\attachments\Côte d'Ivoire_3e0ac251-71a8-488f-8deb-bce2e3ea84e0_Screenshot_20240614-180234.jpg in row 2
Embedding image c:\Users\rami8629\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\Code-Repos\ArcGIS_Code_Repo\src\tests\attachments\Côte d'Ivoire_13b05cad-9f1b-4ad0-8500-620fbbb1fb2a_Screenshot_20240616-180857.jpg in row 3
Embedding image c:\Users\rami8629\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\Code-Repos\ArcGIS_Code_Repo\src\tests\attachments\Côte d'Ivoire_13b05cad-9f1b-4ad0-8500-620fbbb1fb2a_Screenshot_20240615-110620.jpg in row 4
Embedding image c:\Users\rami8629\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\Code-Repos\ArcGIS_Code_Repo\src